In [30]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from operator import add
from functools import reduce
from PIL import Image

In [52]:
image = cv2.imread('square_circuit.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [53]:
# plt.imshow(image, cmap='gray')
image.shape

(2560, 2560)

In [116]:
def intensity_criteria(region):
    """
    Returns True if the standard deviation of the intensity values
    within the region is below a threshold, indicating that the
    region is homogeneous.
    """
    intensity_std = np.std(region)
    threshold = 10  # Adjust this threshold to control the level of segmentation
    return intensity_std < threshold

def checkEquals(myList):
    """
    Returns True if the image can be divided further
    """
    first = myList[0]
    return all((x == first).all() for x in myList)

def is_divisible(region):
    if region.shape == (1,0) or region.shape == (0,1):
        return False
    return region.shape != (1,1)

def split4(region):
    """
    Splits the given region in 4 equal parts
    """
    half_split = np.array_split(region, 2)
    res = map(lambda x: np.array_split(x, 2, axis=1), half_split)
    return reduce(add, res)

In [117]:
intensity_criteria(image)

False

In [121]:
arr = np.array([[1,1,4,5,6],
                [1,1,4,5,6],
                [1,1,4,5,6],
                [1,1,4,5,6],
                [1,1,4,5,6]])
split_arr = split4(arr)
a,b,c,d = split_arr

split_arr2 = split4(a)
a2,b2,c2,d2 = split_arr2

print(is_divisible(c2))
print(split_arr)
print(split_arr2)
# print(split4(c2))
print(intensity_criteria(arr))

True
[array([[1, 1, 4],
       [1, 1, 4],
       [1, 1, 4]]), array([[5, 6],
       [5, 6],
       [5, 6]]), array([[1, 1, 4],
       [1, 1, 4]]), array([[5, 6],
       [5, 6]])]
[array([[1, 1],
       [1, 1]]), array([[4],
       [4]]), array([[1, 1]]), array([[4]])]
True


In [157]:
class QuadTree:
    def __init__(self, image):
        self.children = [None, None, None, None]
        self.image_matrix = image
        self.is_homogeneous = intensity_criteria(image)
        self.region_area = (image.shape[0], image.shape[1])
        self.is_leaf = True # TBD
        self.split(image)
    
    def merge(self):
        pass

    def split(self, image):
        if is_divisible(image) and self.is_homogeneous == False:
        # if not checkEquals(image) and self.is_homogeneous == False:
            north_west, north_east, south_west, south_east = split4(image)
            # print(north_west.shape)
            self.children[0] = QuadTree(north_west)
            self.children[1] = QuadTree(north_east)
            self.children[2] = QuadTree(south_west)
            self.children[3] = QuadTree(south_east)

In [158]:
quadtree = QuadTree(image)

c:\Users\roroa\anaconda3\envs\py38\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\roroa\anaconda3\envs\py38\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\roroa\anaconda3\envs\py38\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [161]:
quadtree.children[0].children[0].children[0].children[0].children[0].children[0].children[0].image_matrix

array([[167, 167, 167, 167, 167, 167, 167, 168, 170, 171, 172, 173, 173,
        174, 175, 175, 175, 175, 174, 171],
       [167, 167, 167, 167, 166, 167, 165, 167, 169, 170, 171, 172, 172,
        173, 173, 173, 174, 174, 173, 169],
       [165, 165, 165, 165, 165, 164, 164, 165, 166, 167, 168, 170, 170,
        170, 171, 171, 171, 171, 170, 169],
       [164, 164, 164, 164, 163, 163, 162, 164, 165, 166, 167, 168, 169,
        169, 169, 169, 169, 169, 169, 168],
       [164, 164, 164, 163, 163, 163, 163, 164, 166, 166, 168, 168, 168,
        170, 170, 171, 171, 170, 169, 169],
       [164, 164, 163, 163, 163, 163, 164, 164, 165, 167, 168, 169, 170,
        171, 172, 172, 172, 171, 170, 169],
       [163, 163, 163, 162, 163, 163, 164, 164, 166, 168, 169, 170, 171,
        172, 173, 173, 174, 173, 172, 172],
       [162, 162, 161, 162, 164, 163, 164, 165, 167, 168, 170, 172, 173,
        174, 174, 175, 175, 175, 175, 174],
       [160, 160, 161, 161, 162, 162, 163, 164, 164, 167, 169, 1